# Code 3: Interactive Annotation Tool for Detecting Destroyed Buildings in Images

This script serves as an interactive annotation tool designed specifically for identifying and annotating regions of interest containing destroyed buildings within images. It employs the matplotlib library to create an interactive environment that enables users to draw bounding boxes around the areas of images where destroyed buildings are present. These bounding boxes serve as annotations for machine learning tasks such as object detection. The script generates XML annotations for each bounding box, encapsulating essential information about the destroyed building's location and image attributes. This tool significantly streamlines the process of generating labeled data for training models to detect and analyze destroyed buildings within images.

**Import necessary libraries**

In [ ]:
# Add this line at the beginning of your Jupyter Notebook cell
%matplotlib notebook

import os
import cv2
import matplotlib.pyplot as plt
from matplotlib.widgets import RectangleSelector

import xml.etree.ElementTree as ET
from xml.dom import minidom

**Definition**

In [ ]:

def line_select_callback(clk, rls):
    global tl_list
    global br_list
    tl_list.append((int(clk.xdata), int(clk.ydata)))
    br_list.append((int(rls.xdata), int(rls.ydata)))


def toggle_selector(event):
    toggle_selector.RS.set_active(True)


def onkeypress(event):
    global tl_list
    global br_list
    if event.key == 'q':
        generate_xml(tl_list, br_list, file_name, name_class)
        tl_list = []
        br_list = []


def generate_xml(tl_list, br_list, file_name, name_class):
    annotation = ET.Element("annotation")

    filename = ET.SubElement(annotation, "filename")
    filename.text = file_name

    size = ET.SubElement(annotation, "size")
    width = ET.SubElement(size, "width")
    height = ET.SubElement(size, "height")
    depth = ET.SubElement(size, "depth")
    width.text = str(abs(tl_list[0][0] - br_list[0][0]))
    height.text = str(abs(tl_list[0][1] - br_list[0][1]))
    depth.text = "3"  # Assuming RGB images

    object_elem = ET.SubElement(annotation, "object")
    name_elem = ET.SubElement(object_elem, "name")
    name_elem.text = name_class

    bndbox = ET.SubElement(object_elem, "bndbox")
    xmin = ET.SubElement(bndbox, "xmin")
    ymin = ET.SubElement(bndbox, "ymin")
    xmax = ET.SubElement(bndbox, "xmax")
    ymax = ET.SubElement(bndbox, "ymax")
    xmin.text = str(tl_list[0][0])
    ymin.text = str(tl_list[0][1])
    xmax.text = str(br_list[0][0])
    ymax.text = str(br_list[0][1])

    xml_str = ET.tostring(annotation, encoding="unicode")
    dom = minidom.parseString(xml_str)
    pretty_xml_str = dom.toprettyxml()

    xml_filename = os.path.join('annotations path', file_name.replace('.jpg', '.xml'))
    with open(xml_filename, "w") as xml_file:
        xml_file.write(pretty_xml_str)

**Create Annotations**

In [ ]:
# Main
image_folder = ''
file_name = ''
name_class = ''

tl_list = []
br_list = []
file_names = os.listdir(image_folder)

for file_name in file_names:
    if file_name[0] != '.':
        name_class, sep, tail = file_name.partition('_')
        dir_file = os.path.join(image_folder, file_name)

        fig, ax = plt.subplots(1)
        image = cv2.imread(dir_file)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        ax.imshow(image)

        toggle_selector.RS = RectangleSelector(
            ax, line_select_callback,
            useblit=True,
            button=[1], minspanx=5, minspany=5,
            spancoords='pixels', interactive=True
        )

        bbox = plt.connect('key_press_event', toggle_selector)
        key = plt.connect('key_press_event', onkeypress)
        plt.show()

print('Number of Processed Images:', len(file_names))
